###### Company Introduction
Your client for this project is a retail banking institution.

They are going to float a stock trading facility for their existing customers.
The idea is to use data to classify whether a customer belongs to a high net worth or low net worth group.
They will have to incentivize their customers to adopt their offerings.
One way to incentivize is to offer discounts on the commission for trading transactions.

Current Scenario
The company rolled out this service to about 10,000+ of its customers and observed their trading behavior for 6 months and after that, they labeled them into two revenue grids 1 and 2.

In [86]:
import numpy as np                     

import pandas as pd
pd.set_option('mode.chained_assignment', None)      # To suppress pandas warnings.
pd.set_option('display.max_colwidth', None)         # To display all the data in each column
pd.set_option('display.max_columns', None)          # To display every column of the dataset in head()

import warnings
warnings.filterwarnings('ignore')                   # To suppress all the warnings in the notebook.

In [87]:
cust=pd.read_csv('existing_base_train.csv')

In [88]:
cust.shape

(8124, 32)

In [89]:
cust.Revenue_Grid.value_counts()

2    7264
1     860
Name: Revenue_Grid, dtype: int64

# Feature Engineering 

In [90]:
def home_stat(g): 
    if g =='Own Home':
        return 'Owner'
    elif g == 'Rent Privately':
        return 'Rent'
    elif g == 'Rent from Council/HA':
        return 'Rent'
    elif g == 'Live in Parental Hom':
        return 'Rent'
    elif g in 'Unclassified':
        return 'Rent'
working=['Professional','Manual Worker','Business Manager','Secretarial/Admin','Other']
non_working=['Retired','Housewife','Student']
def occ(g):
    if (g.occupation in working) & (g.occupation_partner in working):
        return 'both_working'
    elif (g.occupation in non_working) & (g.occupation_partner in non_working):
        return 'both_non_working'
    else:
        return 'single'
def self_emp(g):
    if (g.self_employed == 'Yes') & (g.self_employed_partner == 'Yes'):
        return 'both_yes'
    elif (g.self_employed == 'No') & (g.self_employed_partner == 'No'):
        return 'both_no'
    else:
        return 'single'

def feature_engg():
    cust['children']=cust['children'].replace('Zero','0')
    cust['children']=cust['children'].replace('4+','4')
    cust['family_income']=cust['family_income'].replace(['>=35,000','<22,500, >=20,000','<25,000, >=22,500','<30,000, >=27,500', '<20,000, >=17,500','<27,500, >=25,000','< 4,000', '<15,000, >=12,500', '<17,500, >=15,000','< 8,000, >= 4,000', '<12,500, >=10,000', '<10,000, >= 8,000'],['35000+','20,000-22,500','22,500-25,000','27,500-27,500','17,500-20,000','25,000-27,500','0-4,000','12,500-15,000','15,000-17,500','4,000-8,000','10,000-12,500','8,000-10,000'])
    cust['total_invest']=cust.Investment_in_Mutual_Fund+cust.Investment_Tax_Saving_Bond+cust.Portfolio_Balance
    cust['total_loan']=cust.Personal_Loan+cust.Home_Loan
    cust['total_isurance']=cust.Life_Insurance+cust.Medical_Insurance
    cust['home_status_new']=cust['home_status'].apply(home_stat)
    cust['occupation_Status_new']=cust.apply(occ,axis=1)
    cust['self_employed_new']=cust.apply(self_emp,axis=1)
feature_engg()
cust.shape

(8124, 38)

In [91]:
cust.shape

(8124, 38)

In [92]:
cust_new=cust[['REF_NO','children','age_band','status','family_income','post_area','Average_Credit_Card_Transaction','Balance_Transfer','Term_Deposit', 'total_isurance','Average_A/C_Balance','Online_Purchase_Amount','gender', 'region','total_invest', 'total_loan', 'home_status_new','occupation_Status_new', 'self_employed_new','Revenue_Grid']]

# One hot encoding

In [93]:
def encoding_stat(cust_new):
    stat=pd.get_dummies(cust_new.status)
    stat.head(10)
    stat.drop('Unknown',axis=1,inplace=True)
    cust_new=pd.concat([cust_new,stat],axis=1)
    return cust_new
cust_new=encoding_stat(cust_new)



In [94]:
cust_new.shape

(8124, 24)

In [95]:
cust_new.columns

Index(['REF_NO', 'children', 'age_band', 'status', 'family_income',
       'post_area', 'Average_Credit_Card_Transaction', 'Balance_Transfer',
       'Term_Deposit', 'total_isurance', 'Average_A/C_Balance',
       'Online_Purchase_Amount', 'gender', 'region', 'total_invest',
       'total_loan', 'home_status_new', 'occupation_Status_new',
       'self_employed_new', 'Revenue_Grid', 'Divorced/Separated', 'Partner',
       'Single/Never Married', 'Widowed'],
      dtype='object')

In [96]:
def encoding(cust_new):
    cat_columns=['home_status_new','occupation_Status_new','self_employed_new']
    cat_dum=pd.get_dummies(cust_new[cat_columns],drop_first=True)
    cust_new=pd.concat([cust_new,cat_dum],axis=1)
    return cust_new
cust_new=encoding(cust_new)

In [97]:
cust_new.columns

Index(['REF_NO', 'children', 'age_band', 'status', 'family_income',
       'post_area', 'Average_Credit_Card_Transaction', 'Balance_Transfer',
       'Term_Deposit', 'total_isurance', 'Average_A/C_Balance',
       'Online_Purchase_Amount', 'gender', 'region', 'total_invest',
       'total_loan', 'home_status_new', 'occupation_Status_new',
       'self_employed_new', 'Revenue_Grid', 'Divorced/Separated', 'Partner',
       'Single/Never Married', 'Widowed', 'home_status_new_Rent',
       'occupation_Status_new_both_working', 'occupation_Status_new_single',
       'self_employed_new_both_yes', 'self_employed_new_single'],
      dtype='object')

In [98]:
cust_new.shape

(8124, 29)

In [99]:
cust_new.columns

Index(['REF_NO', 'children', 'age_band', 'status', 'family_income',
       'post_area', 'Average_Credit_Card_Transaction', 'Balance_Transfer',
       'Term_Deposit', 'total_isurance', 'Average_A/C_Balance',
       'Online_Purchase_Amount', 'gender', 'region', 'total_invest',
       'total_loan', 'home_status_new', 'occupation_Status_new',
       'self_employed_new', 'Revenue_Grid', 'Divorced/Separated', 'Partner',
       'Single/Never Married', 'Widowed', 'home_status_new_Rent',
       'occupation_Status_new_both_working', 'occupation_Status_new_single',
       'self_employed_new_both_yes', 'self_employed_new_single'],
      dtype='object')

In [100]:
cust_new.drop(['home_status_new','occupation_Status_new','self_employed_new','status'],axis=1,inplace=True)

In [101]:
cust_new.gender.value_counts()

Female     6106
Male       1987
Unknown      31
Name: gender, dtype: int64

In [102]:
cust_new.head()

,REF_NO,children,age_band,family_income,post_area,Average_Credit_Card_Transaction,Balance_Transfer,Term_Deposit,total_isurance,Average_A/C_Balance,Online_Purchase_Amount,gender,region,total_invest,total_loan,Revenue_Grid,Divorced/Separated,Partner,Single/Never Married,Widowed,home_status_new_Rent,occupation_Status_new_both_working,occupation_Status_new_single,self_employed_new_both_yes,self_employed_new_single
0,5466,2,31-35,35000+,M51,26.98,29.99,312.25,388.51,108.85,7.68,Female,North West,503.70,230.87,2,0,1,0,0,0,1,0,0,0
1,9091,0,45-50,35000+,L40,35.98,74.48,0.00,110.95,48.45,18.99,Female,North West,89.22,15.99,2,0,1,0,0,0,1,0,0,0
2,9744,1,36-40,"20,000-22,500",TA19,0.00,24.46,0.00,18.44,0.00,0.00,Female,South West,24.96,0.02,2,0,1,0,0,1,1,0,1,0
3,10700,2,31-35,"22,500-25,000",FK2,44.99,0.00,0.00,29.99,0.00,0.00,Female,Scotland,68.98,0.00,2,0,1,0,0,0,1,0,0,0
4,1987,0,55-60,35000+,LS23,0.00,0.00,0.00,0.00,0.00,0.00,Female,Unknown,11.86,0.00,2,0,1,0,0,0,0,1,0,0


In [103]:
def encoding_gender(cust_new):
    gender=pd.get_dummies(cust_new.gender)
    #gender.head(10)
    gender.drop('Unknown',axis=1,inplace=True)
    cust_new=pd.concat([cust_new,gender],axis=1)
    return cust_new
cust_new=encoding_gender(cust_new)

In [104]:
cust_new.shape

(8124, 27)

In [105]:
cust_new.columns

Index(['REF_NO', 'children', 'age_band', 'family_income', 'post_area',
       'Average_Credit_Card_Transaction', 'Balance_Transfer', 'Term_Deposit',
       'total_isurance', 'Average_A/C_Balance', 'Online_Purchase_Amount',
       'gender', 'region', 'total_invest', 'total_loan', 'Revenue_Grid',
       'Divorced/Separated', 'Partner', 'Single/Never Married', 'Widowed',
       'home_status_new_Rent', 'occupation_Status_new_both_working',
       'occupation_Status_new_single', 'self_employed_new_both_yes',
       'self_employed_new_single', 'Female', 'Male'],
      dtype='object')

In [106]:
cust_new.drop('gender',axis=1,inplace=True)

In [107]:
cust_new.columns

Index(['REF_NO', 'children', 'age_band', 'family_income', 'post_area',
       'Average_Credit_Card_Transaction', 'Balance_Transfer', 'Term_Deposit',
       'total_isurance', 'Average_A/C_Balance', 'Online_Purchase_Amount',
       'region', 'total_invest', 'total_loan', 'Revenue_Grid',
       'Divorced/Separated', 'Partner', 'Single/Never Married', 'Widowed',
       'home_status_new_Rent', 'occupation_Status_new_both_working',
       'occupation_Status_new_single', 'self_employed_new_both_yes',
       'self_employed_new_single', 'Female', 'Male'],
      dtype='object')

In [108]:
#label encoding for age_band
ageband_map={
                'Unknown':0, '18-21':1, '22-25':2,'26-30':3, '31-35':4,'36-40':5,'41-45':6, '45-50':7,
                 '51-55':8,'55-60':9, '61-65':10, '65-70':11,'71+':12 
}
cust_new['ageband_ordinal']=cust_new.age_band.map(ageband_map)

In [109]:
cust_new.drop('age_band',axis=1,inplace=True)

In [110]:
#label encoding for age_band
family_income_map={
                'Unknown':0, '0-4,000':1, '4,000-8,000':2,'8,000-10,000':3, '10,000-12,500':4,'12,500-15,000':5,'15,000-17,500':6, '17,500-20,000':7,
                 '20,000-22,500':8,'22,500-25,000':9, '25,000-27,500':10, '27,500-27,500':11,'35000+':12 
}
cust_new['family_income_ordinal']=cust_new.family_income.map(family_income_map)

In [111]:
cust_new.drop('family_income',axis=1,inplace=True)

In [112]:
#we can replace with its value counts as no one is same number
cust_new.region.value_counts()

South East          1680
North West          1517
Unknown              866
South West           769
West Midlands        658
East Midlands        623
Scotland             615
North                460
Wales                437
East Anglia          344
Northern Ireland     135
Isle of Man           15
Channel Islands        5
Name: region, dtype: int64

In [113]:
reg=cust_new.groupby('region').size()/len(cust_new)
cust_new.loc[:,'region_encoded']=cust_new.region.map(reg)
# try with just value count if F1 changes

In [114]:
cust_new.drop('post_area',axis=1,inplace=True)

In [115]:
cust_new.drop('region',axis=1,inplace=True)

In [116]:
cust_new.columns

Index(['REF_NO', 'children', 'Average_Credit_Card_Transaction',
       'Balance_Transfer', 'Term_Deposit', 'total_isurance',
       'Average_A/C_Balance', 'Online_Purchase_Amount', 'total_invest',
       'total_loan', 'Revenue_Grid', 'Divorced/Separated', 'Partner',
       'Single/Never Married', 'Widowed', 'home_status_new_Rent',
       'occupation_Status_new_both_working', 'occupation_Status_new_single',
       'self_employed_new_both_yes', 'self_employed_new_single', 'Female',
       'Male', 'ageband_ordinal', 'family_income_ordinal', 'region_encoded'],
      dtype='object')

In [117]:
independent_cols=['REF_NO', 'children', 'Average_Credit_Card_Transaction',
       'Balance_Transfer', 'Term_Deposit', 'total_isurance',
       'Average_A/C_Balance', 'Online_Purchase_Amount', 'total_invest',
       'total_loan','Divorced/Separated', 'Partner',
       'Single/Never Married', 'Widowed', 'home_status_new_Rent',
       'occupation_Status_new_both_working', 'occupation_Status_new_single',
       'self_employed_new_both_yes', 'self_employed_new_single', 'Female',
       'Male', 'ageband_ordinal', 'family_income_ordinal']# removed , 'region_encoded'
X=cust_new[independent_cols] 
y=cust_new.Revenue_Grid

In [118]:
X.shape

(8124, 23)

In [119]:
y.shape

(8124,)

In [120]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### Scaling for Logistic Regression

In [121]:
from sklearn.preprocessing import RobustScaler
robust = RobustScaler()
X_train[independent_cols] = robust.fit_transform(X_train)
X_test[independent_cols] = robust.transform(X_test)

### Logistic Regression Model creation

In [122]:
from sklearn.linear_model import LogisticRegression 
logreg = LogisticRegression()
logreg.fit(X_train,y_train)

LogisticRegression()

## Prediction

In [123]:
#predicting on train data
y_pred_train = logreg.predict(X_train)

#predicting on test data
y_pred_test = logreg.predict(X_test)

### Model Evaluation

In [124]:
from sklearn.metrics import accuracy_score   
pred = logreg.predict(X_test)
print('Accuracy score for test data is:', accuracy_score(y_test,pred))

Accuracy score for test data is: 0.9372307692307692


In [125]:
from sklearn.metrics import f1_score    
f1_score(y_test,y_pred_test)

0.6304347826086956

In [126]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           1       0.82      0.51      0.63       170
           2       0.95      0.99      0.97      1455

    accuracy                           0.94      1625
   macro avg       0.88      0.75      0.80      1625
weighted avg       0.93      0.94      0.93      1625



# Test data 

In [127]:
final_test=pd.read_csv('existing_base_test.csv')

In [128]:
#y_pred_final=model.predict(final_test)
def home_stat(g):
    if g =='Own Home':
        return 'Owner'
    elif g == 'Rent Privately':
        return 'Rent'
    elif g == 'Rent from Council/HA':
        return 'Rent'
    elif g == 'Live in Parental Hom':
        return 'Rent'
    elif g in 'Unclassified':
        return 'Rent'
working=['Professional','Manual Worker','Business Manager','Secretarial/Admin','Other']
non_working=['Retired','Housewife','Student']
def occ(g):
    if (g.occupation in working) & (g.occupation_partner in working):
        return 'both_working'
    elif (g.occupation in non_working) & (g.occupation_partner in non_working):
        return 'both_non_working'
    else:
        return 'single'
def self_emp(g):
    if (g.self_employed == 'Yes') & (g.self_employed_partner == 'Yes'):
        return 'both_yes'
    elif (g.self_employed == 'No') & (g.self_employed_partner == 'No'):
        return 'both_no'
    else:
        return 'single'


def feature_engg():
    final_test['children']=final_test['children'].replace('Zero','0')
    final_test['children']=final_test['children'].replace('4+','4')
    final_test['family_income']=final_test['family_income'].replace(['>=35,000','<22,500, >=20,000','<25,000, >=22,500','<30,000, >=27,500', '<20,000, >=17,500','<27,500, >=25,000','< 4,000', '<15,000, >=12,500', '<17,500, >=15,000','< 8,000, >= 4,000', '<12,500, >=10,000', '<10,000, >= 8,000'],['35000+','20,000-22,500','22,500-25,000','27,500-27,500','17,500-20,000','25,000-27,500','0-4,000','12,500-15,000','15,000-17,500','4,000-8,000','10,000-12,500','8,000-10,000'])
    final_test['total_invest']=final_test.Investment_in_Mutual_Fund+final_test.Investment_Tax_Saving_Bond+final_test.Portfolio_Balance
    final_test['total_loan']=final_test.Personal_Loan+final_test.Home_Loan
    final_test['total_isurance']=final_test.Life_Insurance+final_test.Medical_Insurance
    final_test['home_status_new']=final_test['home_status'].apply(home_stat)
    final_test['occupation_Status_new']=final_test.apply(occ,axis=1)
    final_test['self_employed_new']=final_test.apply(self_emp,axis=1)
feature_engg()
final_test.shape


(2031, 37)

In [129]:
final_test_new=final_test[['REF_NO','children','age_band','status','family_income','post_area','Average_Credit_Card_Transaction','Balance_Transfer','Term_Deposit', 'total_isurance','Average_A/C_Balance','Online_Purchase_Amount','gender', 'region','total_invest', 'total_loan', 'home_status_new','occupation_Status_new', 'self_employed_new']]

In [130]:
def encoding_stat(final_test_new):
    stat=pd.get_dummies(final_test_new.status)
    stat.head(10)
    stat.drop('Unknown',axis=1,inplace=True)
    final_test_new=pd.concat([final_test_new,stat],axis=1)
    return final_test_new
final_test_new=encoding_stat(final_test_new)
def encoding(final_test_new):
    cat_columns=['home_status_new','occupation_Status_new','self_employed_new']
    cat_dum=pd.get_dummies(final_test_new[cat_columns],drop_first=True)
    final_test_new=pd.concat([final_test_new,cat_dum],axis=1)
    return final_test_new
final_test_new=encoding(final_test_new)
final_test_new.drop(['home_status_new','occupation_Status_new','self_employed_new','status'],axis=1,inplace=True)
def encoding_gender(final_test_new):
    gender=pd.get_dummies(final_test_new.gender)
    #gender.head(10)
    gender.drop('Unknown',axis=1,inplace=True)
    final_test_new=pd.concat([final_test_new,gender],axis=1)
    return final_test_new
final_test_new=encoding_gender(final_test_new)
final_test_new.drop('gender',axis=1,inplace=True)#label encoding for age_band
ageband_map={
                'Unknown':0, '18-21':1, '22-25':2,'26-30':3, '31-35':4,'36-40':5,'41-45':6, '45-50':7,
                 '51-55':8,'55-60':9, '61-65':10, '65-70':11,'71+':12 
}
final_test_new['ageband_ordinal']=final_test_new.age_band.map(ageband_map)
final_test_new.drop('age_band',axis=1,inplace=True)
#label encoding for family income
family_income_map={
                'Unknown':0, '0-4,000':1, '4,000-8,000':2,'8,000-10,000':3, '10,000-12,500':4,'12,500-15,000':5,'15,000-17,500':6, '17,500-20,000':7,
                 '20,000-22,500':8,'22,500-25,000':9, '25,000-27,500':10, '27,500-27,500':11,'35000+':12 
}
final_test_new['family_income_ordinal']=final_test_new.family_income.map(family_income_map)
final_test_new.drop('family_income',axis=1,inplace=True)
#reg=final_test_new.groupby('region').size()/len(final_test_new)
#final_test_new.loc[:,'region_encoded']=final_test_new.region.map(reg)
independent_cols=['REF_NO', 'children', 'Average_Credit_Card_Transaction',
       'Balance_Transfer', 'Term_Deposit', 'total_isurance',
       'Average_A/C_Balance', 'Online_Purchase_Amount', 'total_invest',
       'total_loan', 'Divorced/Separated', 'Partner',
       'Single/Never Married', 'Widowed', 'home_status_new_Rent',
       'occupation_Status_new_both_working', 'occupation_Status_new_single',
       'self_employed_new_both_yes', 'self_employed_new_single', 'Female',
       'Male', 'ageband_ordinal', 'family_income_ordinal']
X=final_test_new[independent_cols]

In [131]:
X.shape

(2031, 23)

In [132]:
X.columns

Index(['REF_NO', 'children', 'Average_Credit_Card_Transaction',
       'Balance_Transfer', 'Term_Deposit', 'total_isurance',
       'Average_A/C_Balance', 'Online_Purchase_Amount', 'total_invest',
       'total_loan', 'Divorced/Separated', 'Partner', 'Single/Never Married',
       'Widowed', 'home_status_new_Rent', 'occupation_Status_new_both_working',
       'occupation_Status_new_single', 'self_employed_new_both_yes',
       'self_employed_new_single', 'Female', 'Male', 'ageband_ordinal',
       'family_income_ordinal'],
      dtype='object')

In [133]:
from sklearn.preprocessing import RobustScaler
robust = RobustScaler()
X[independent_cols] = robust.fit_transform(X)
#X_test[independent_cols] = robust.transform(X_test)

In [134]:
y_pred_final=logreg.predict(X)

In [135]:
df3 = pd.DataFrame({"Ref_no":final_test_new.REF_NO,"Revenue_grid":y_pred_final})

In [136]:
df3.to_csv('cust_classification_output.csv',index=False, header=False)

In [137]:
df3.Revenue_grid.value_counts()

2    1925
1     106
Name: Revenue_grid, dtype: int64